### Namibian power network generated from PyPSA-Earth-Sec - an open energy system optimization model

scenario configuration options : optimistic, realistic, conservative

References : 

For more information on the model, refer to the following sources.

[pypsa-earth-sec](https://github.com/pypsa-meets-earth/pypsa-earth-sec) on GitHub.

check [documentation](https://github.com/pypsa-meets-earth/documentation) for [pypsa-earth](https://github.com/pypsa-meets-earth/pypsa-earth) relation information.

see [pypsa-meets-earth](https://github.com/pypsa-meets-earth), 2023) project on GitHub. 


### Import packages

In [ ]:
#Import packages
import os
import sys
import logging
import pypsa
import yaml
import pandas as pd
import geopandas as gpd
import numpy as np
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io.img_tiles import OSM
import cartopy.feature as cfeature

from scipy.sparse import csgraph
from itertools import product
import re

from shapely.geometry import Point, LineString
import shapely, shapely.prepared
from shapely.wkt import loads
from shapely.validation import make_valid

logger = logging.getLogger(__name__)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 70)
#for solved network plot

from matplotlib.patches import Circle, Ellipse
from matplotlib.legend_handler import HandlerPatch
from pypsa.plot import add_legend_circles, add_legend_lines, add_legend_patches
plt.style.use("bmh")

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid", font_scale=1)

### Node query

In [ ]:
n = pypsa.Network("/home/raj08555/dev/sandbox-cs/pypsa-earth-sec/results/MT_real30/postnetworks/elec_s_30_ec_lc1.0_Co2L_12H_2030_0.082_AP_538export.nc") # keep the .nc file in same directory as the script

c = n.buses
g = n.generators
g_t = n.generators_t



In [ ]:
nam=pd.read_csv("/home/raj08555/dev/thesis-notebook/analysis/Namibia_bus_region_constituencies.csv")
#nam.set_index("Bus name")
nam

In [ ]:
n.stores_t['e']['co2 atmosphere'].max()

In [ ]:
%run -i ../functions/helper_functions.ipynb
n.links.drop(
        n.links.index[(n.links.carrier != "DC") & (n.links.carrier != "B2B")],
        inplace=True,
    )
#plot_network(n)
#n.plot()

In [ ]:
#n.carriers

### maximum installable capacity

In [ ]:

def high_node_cap(n, tech):
    s_max = n.generators.filter(regex='{}$'.format(tech), axis=0).p_nom_max.max()
    bus = n.generators.filter(regex='{}$'.format(tech), axis=0).loc[n.generators.filter(regex='{}$'.format(tech), axis=0).p_nom_max==s_max]
    print("\n Highest maximum installable capacity node for {}:".format(tech))
    bus["region"] =nam[nam["bus_region"] == bus.bus.values[0]]["name"].values[0]
    bus["p_nom_max"] = bus["p_nom_max"]*1e-3
    return bus[["bus", "p_nom_max", "region"]]

In [ ]:
high_node_cap(n, "solar")

In [ ]:
high_node_cap(n, "wind")

In [ ]:
high_node_cap(n, "ror")

### Maximum installed capacity

In [ ]:
def high_node_in(n, tech):
    s_max = n.generators.filter(regex='{}$'.format(tech), axis=0).p_nom_opt.max()
    bus = n.generators.filter(regex='{}$'.format(tech), axis=0).loc[n.generators.filter(regex='{}$'.format(tech), axis=0).p_nom_opt==s_max]
    print("\n Maximum installed capacity node:")
    bus["region"] = nam[nam["bus_region"] == bus.bus.values[0]]["name"].values[0]
    bus["p_nom_opt"] = bus["p_nom_opt"]*1e-3
    print(str(bus.bus.values))
    return(bus[["bus", "p_nom_opt", "region"]])

In [ ]:
high_node_in(n, "solar")

In [ ]:
high_node_in(n, "wind")

In [ ]:
timestep=12

In [ ]:

# =============================================================================
# #Max Generated solar power
# =============================================================================
def generated_node(n, tech):
      #p_max_pu = g_t['p']
      bus = n.generators_t.p.filter(regex='{}$'.format(tech), axis=1).sum(axis=0)
      busid = n.generators_t.p.filter(regex='{}$'.format(tech), axis=1).sum(axis=0).idxmax()
      busgen= n.generators_t.p.filter(regex='{}$'.format(tech), axis=1).sum(axis=0).sum(axis=0).max()*timestep * 1e-6
      return bus #nam[nam["bus_region"] == str.strip(busid)]["name"].values]
      # bus["region"] = nam[nam["bus_region"] == bus.bus.values[0]]["name"].values[0]
      # print('\n \n maximum solar energy is generated at node \n ' + 
      #       str(bus.sum(axis=0).idxmax()) + ' is ' + str(bus.sum(axis=0).max()*24) + ' MWh')

In [ ]:
generated_node(n, "solar")

In [ ]:
# =============================================================================
# #Max generated wind power
# =============================================================================
p_max_pu = g_t['p']
y = p_max_pu.filter(regex='wind$', axis=1)
print('\n \n maximum wind energy generated at node \n' + 
      str(y.sum(axis=0).idxmax()) + ' is ' + str(y.sum(axis=0).max()*24) + ' MWh')

### Timestep Query

In [ ]:

# =============================================================================
# # timestep with highest solar generation
# =============================================================================
solar_gen_max = g_t['p'].filter(regex='solar$', axis=1).max()
solar_gen_max_t = g_t['p'].filter(like=solar_gen_max.idxmax(), axis=1).idxmax()
print("\n \n Highest solar generation timestep: \n")
print(str(solar_gen_max_t[0]) +"   "+ str(solar_gen_max.max()*24) + " MWh")

# =============================================================================
# # timestep with lowest solar generation
# =============================================================================
solar_gen_min = g_t['p'].filter(regex='solar$', axis=1).min()
solar_gen_min_t = g_t['p'].filter(like=solar_gen_min.idxmin(), axis=1).idxmin()
print("\n \n Lowest solar generation timestep: \n")
print(str(solar_gen_min_t[0]) +"   "+ str(solar_gen_min.min()*24) + " MWh")


In [ ]:

# =============================================================================
# # timeste with highest wind capacity
# =============================================================================
# =============================================================================
wind_gen_max = g_t['p'].filter(regex='wind$', axis=1).max()
wind_gen_max_t = g_t['p'].filter(like=wind_gen_max.idxmax(), axis=1).idxmax()
print("\n \n Highest wind generation timestep: \n")
print(str(wind_gen_max_t[0]) +"   "+ str(wind_gen_max.max()*24) + " MWh")

# =============================================================================
# # timestep with lowest wind generation
# =============================================================================
wind_gen_min = g_t['p'].filter(regex='wind$', axis=1).min()
wind_gen_min_t = g_t['p'].filter(like=wind_gen_min.idxmin(), axis=1).idxmin()
print("\n \n Lowest solar generation timestep: \n")
print(str(wind_gen_min_t[0]) +"   "+ str(wind_gen_min.min()*24) + " MWh")

In [ ]:
#=============================================================================

# =============================================================================
# # timestep with highest demand overall
# =============================================================================
load = n.loads_t['p']
load_max = load.filter(like=load.max().idxmax(), axis=1)

print("\n \n highest demand overall timestep:")
print(str(load_max.idxmax().max()) +"   " + str(load_max.max().max()*24) + ' MWh')
#=============================================================================

# =============================================================================
# # timestep with lowest demand overall
# =============================================================================
load_min = load.filter(like=load.min().idxmin(), axis=1)

print("\n \n lowest demand overall timestep:")
print(str(load_min.idxmin().min()) +"   " + str(load_min.min().min()*24) + ' MWh')


### Node-demand

In [ ]:

# =============================================================================
# #Node with Largest electrolyzer capacity
# =============================================================================

print("\n \n Largest electrolyzer capacity node:")

print(n.stores.filter(like='H2', axis=0)['e_nom_opt'].idxmax())
print(n.stores.filter(like='H2', axis=0)['e_nom_opt'].max())

# =============================================================================
# #Node with Smallest electrolyzer capacity
# =============================================================================

print("\n \n Smallest electrolyzer capacity node:")

print(n.stores.filter(like='H2', axis=0)['e_nom_opt'].idxmin())
print(n.stores.filter(like='H2', axis=0)['e_nom_opt'].min())


In [ ]:
n.stores.filter(like='H2', axis=0)['e_nom_opt']

In [ ]:

# =============================================================================
# #Node with Largest direct air capture capacity
# =============================================================================

print("\n \n Largest direct air capture capacity node:")

print(n.stores.filter(like='co2 stor', axis=0)['e_nom_opt'].idxmax())
print(n.stores.filter(like='co2 stor', axis=0)['e_nom_opt'].max())

# =============================================================================
# #Node with Smallest direct air capture capacity
# =============================================================================

print("\n \n Smallest direct air capture capacity node:")

print(n.stores.filter(like='co2 stor', axis=0)['e_nom_opt'].idxmin())
print(n.stores.filter(like='co2 stor', axis=0)['e_nom_opt'].min())


In [ ]:

# =============================================================================
# #Node with Largest overall land transport demand (EV)
# =============================================================================
load = n.loads_t['p']
print("\n \n Largest overall land transport demand (EV) node:")
print(load.filter(like='land transport EV', axis=1).sum().idxmax())
print(load.filter(like='land transport EV', axis=1).sum().max()*24)


In [ ]:

# =============================================================================
# #Node with Largest overall land transport demand (Fuel cell) node
# =============================================================================

print("\n \n Largest overall land transport demand (Fuel cell) node:")
#print(load.filter(like='land transport fuel cell', axis=1).sum().idxmax())
print(load.filter(like='land transport fuel cell', axis=1).sum().max()*24)

In [ ]:

# =============================================================================
# #Node with Highest aggregate hydro generation node
# =============================================================================

print("\n \n Highest aggregate hydro generation node")
#print(n.storage_units_t['p'].filter(like='hydro', axis=1).sum().idxmax())
print(n.storage_units_t['p'].filter(like='hydro', axis=1).sum().max()*24)

In [ ]:

# =============================================================================
# #Node with Highest h2 demand for industry
# =============================================================================

print("\n \n Highest h2 demand for industry node:")
print(load.filter(like='H2 for industry', axis=1).sum().idxmax())
print(load.filter(like='H2 for industry', axis=1).sum().max()*24)

## Node pipeline

In [ ]:

# =============================================================================
# #Largest capacity pipeline
# =============================================================================
print("\n \n Largest capacity pipeline:")
print(n.links.filter(like='pipeline', axis=0).p_nom_opt.idxmax())
print(n.links.filter(like='pipeline', axis=0).p_nom_opt.max())

# =============================================================================
# # 3rd Largest capacity pipeline
# =============================================================================
print("\n \n 3rd Largest capacity pipeline:")
print(n.links.filter(like='pipeline', axis=0).nlargest(3,'p_nom_opt').index[2])
print(n.links.filter(like='pipeline', axis=0).nlargest(3,'p_nom_opt').p_nom_opt.iloc[2])

# =============================================================================
# #Lowest capacity pipeline
# =============================================================================
print("\n \n Lowest capacity pipeline:")
print(n.links.filter(like='pipeline', axis=0).p_nom_opt.idxmin())
print(n.links.filter(like='pipeline', axis=0).p_nom_opt.min())
